qtw final

Required
Deliverable:
I want you to minimize my dollar cost on an UNKNOWN dataset
That means one you can never see.  Ever.
Each False Positive costs me $10
Each False Negative costs me $500
True Positives and True Negatives cost me $0


Problem:
Cost-sensitive learning 

cost function:
 Total Cost = 10 * Count of FP + 500 * Count of FN

Goal: minimize False Negative or Type-II error


Precision = TruePositives / (TruePositives + FalsePositives)

Recall = TruePositives / (TruePositives + FalseNegatives)


The intuition for recall is that it is not concerned with false positives and it minimizes false negatives. 

Precision is a metric that calculates the percentage of correct predictions for the positive class. Recall calculates the percentage of correct predictions for the positive class out of all positive predictions that could be made. Maximizing precision will minimize the false-positive errors, whereas maximizing recall will minimize the false-negative errors.


Methods
Metric Choice


 Fbeta 

Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)

When choosing beta in your F-beta score the more you care about recall over precision the higher beta you should choose. For example, with F1 score we care equally about recall and precision with F2 score, recall is twice as important to us.


F50


How to compute:
from sklearn.metrics import fbeta_score
y_pred_class = y_pred_pos > threshold
fbeta_score(y_true, y_pred_class, beta)

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fbeta_score.html


 calculate the f2-measure
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# perfect precision, 50% recall
y_true = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
p = precision_score(y_true, y_pred)
r = recall_score(y_true, y_pred)
f = fbeta_score(y_true, y_pred, beta=2.0)
print('Result: p=%.3f, r=%.3f, f=%.3f' % (p, r, f))



eda


check y distribution

pca and see

xgb f beta
if have time rf

ref
https://machinelearningmastery.com/fbeta-measure-for-machine-learning/
https://stackoverflow.com/questions/41060719/using-evaluation-functions-with-parameter-in-xgboost-f-beta


